In [ ]:
import timm
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Load a pretrained Swin Transformer model
model_name = "swin_base_patch4_window7_224"  # Change model name for other variants
model = timm.create_model(model_name, pretrained=True)

model.set_input_size((1120,896))

# Set the model to evaluation mode
model.eval()
model = model.cuda()

x = torch.randn(1, 3, 1120, 896).cuda()


# Get the features just before the average pooling layer
features = model.forward_features(x)
print(features.shape)



torch.Size([1, 35, 28, 1024])


: 

In [5]:
import timm
swin_models = [model for model in timm.list_models() if 'swin' in model and '224' in model]
print(swin_models)

['swin_base_patch4_window7_224', 'swin_large_patch4_window7_224', 'swin_s3_base_224', 'swin_s3_small_224', 'swin_s3_tiny_224', 'swin_small_patch4_window7_224', 'swin_tiny_patch4_window7_224', 'swinv2_cr_base_224', 'swinv2_cr_base_ns_224', 'swinv2_cr_giant_224', 'swinv2_cr_huge_224', 'swinv2_cr_large_224', 'swinv2_cr_small_224', 'swinv2_cr_small_ns_224', 'swinv2_cr_tiny_224', 'swinv2_cr_tiny_ns_224']


In [25]:
print(model.default_cfg['mean'])
print(model.default_cfg['std'])

# Print the model architecture (optional)
#print(model)

# Example: Prepare an image for inference
# Define input size and transformations
input_size = model.default_cfg['input_size'][-1]  # Model's expected input size
transform = transforms.Compose([
    transforms.Normalize(
        mean=model.default_cfg['mean'],
        std=model.default_cfg['std']
    )
])

# Load an image
image_path = "/home/alalbiol/Data/mamo/DDSM_png_16bit_1152x896/cancers/cancer_01/case0001/C_0001_1.RIGHT_CC.png"  # Path to your image
image = np.array(Image.open(image_path)).astype(np.float32)



print("Image max: ", image.max())
print("Image min: ", image.min())

image = torch.from_numpy(image)
image = image.unsqueeze(0)  # Add batch dimension
image = np.repeat(image, 3, axis=0)  # Repeat the image 3 times to create 3 channels
print(image.shape)



(0.485, 0.456, 0.406)
(0.229, 0.224, 0.225)
Image max:  65535.0
Image min:  0.0
torch.Size([3, 1152, 896])


In [23]:


# Preprocess the image
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

print("input_tensor shape: ", input_tensor.shape)

# Run inference
with torch.no_grad():
    output = model(input_tensor)

# Get predicted class
predicted_class = torch.argmax(output, dim=1).item()
print(f"Predicted class: {predicted_class}")


input_tensor shape:  torch.Size([1, 3, 1152, 896])
Predicted class: 111


In [ ]:
print("input size ", 224*4, 224*5)
print("output size ", 224*4/32, 224*5/32)

# Get the feature maps
import torch.nn as nn


avg1 = nn.AdaptiveAvgPool2d()




input size  896 1120
output size  28.0 35.0


42.0